In [5]:
import pandas as pd
import numpy as np


from tardis.io.config_reader import Configuration

/Users/marcwilliamson/miniconda3/envs/tardis/lib/python3.6/importlib/_bootstrap.py:219: QAWarning: pyne.data is not yet QA compliant.
  return f(*args, **kwds)


In [16]:
def constructGrid(axesdict):
    axes = []
    dim = 1
    for key in axesdict:
        ax = axesdict[key]
        axes.append(ax)
        dim = dim * len(ax)
    axesmesh = np.meshgrid(*axes)
    tmp = np.dstack(axesmesh)
    gridpoints = tmp.reshape((dim, len(axes)), order='F')
    return gridpoints

def setconfigitem(tardisconfig, key, value):
    keyitems = key.split('.')
        val = self.tardisconfig[keyitems[0]]
        for k in keyitems[1:]:
            val = val[k]
        

class tardisGrid:
    
    def __init__(self, config, axesdict):
        try:
            tardis_config = Configuration.from_yaml(config)
        except TypeError:
            tardis_config = Configuration.from_config_dict(config)

        self.tardisconfig = tardis_config
        self.config = config
        self.axesdict = axesdict
        
        gridpoints = constructGrid(self.axesdict)
        self.grid = pd.DataFrame(data=gridpoints, columns=self.axesdict.keys())
        
        return
    
    def getconfigitem(self, key):
        keyitems = key.split('.')
        val = self.tardisconfig[keyitems[0]]
        for k in keyitems[1:]:
            val = val[k]
        return val
    
    
    def tardis_config_from_row(row_index):
        tardis_config = Configuration.from_yaml(self.config)
        for col in self.grid.columns:
            

In [17]:
axesdict = {'model.structure.velocity.start':np.arange(10000,15000,1000), 'model.abundances.He':np.arange(0,1,0.1),
           'model.abundances.H':np.arange(0,1,0.25)}

In [18]:
grid = tardisGrid('test.yml', axesdict)

In [19]:
grid.getconfigitem('model.structure.velocity.start')

<Quantity 10000. km / s>

In [20]:
grid.grid

,model.structure.velocity.start,model.abundances.He,model.abundances.H
0,10000.0,0.0,0.00
1,10000.0,0.1,0.00
2,10000.0,0.2,0.00
3,10000.0,0.3,0.00
4,10000.0,0.4,0.00
5,10000.0,0.5,0.00
6,10000.0,0.6,0.00
7,10000.0,0.7,0.00
8,10000.0,0.8,0.00
9,10000.0,0.9,0.00


In [22]:
grid.grid.columns

Index(['model.structure.velocity.start', 'model.abundances.He',
       'model.abundances.H'],
      dtype='object')

In [26]:
grid.tardisconfig['model']['structure']

{'type': 'specific',
 'velocity': {'start': <Quantity 10000. km / s>,
  'stop': <Quantity 20000. km / s>,
  'num': 20},
 'density': {'type': 'branch85_w7',
  'w7_time_0': <Quantity 0.00023148 d>,
  'w7_rho_0': <Quantity 3.e+29 g / cm3>,
  'w7_v_0': <Quantity 1. km / s>}}

In [30]:
a={'a':{'b':2}}

In [32]:
a['a']['b']

2

In [33]:
l=['a','b']

TypeError: eval() arg 1 must be a string, bytes or code object